In [1]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

from quantfreedom.enums import CandleBodyType
from quantfreedom.helper_funcs import dl_ex_candles

np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

In [2]:
candles = dl_ex_candles(
    exchange='mufex',
    symbol='BTCUSDT',
    timeframe='5m',
    since_datetime=datetime(2024,2,14,9,20),
    until_datetime=datetime(2024,2,25),
)
datetimes = candles[:, CandleBodyType.Timestamp].astype("datetime64[ms]")
open_prices = candles[:, CandleBodyType.Open]
high_prices = candles[:, CandleBodyType.High]
low_prices = candles[:, CandleBodyType.Low]
closing_prices = candles[:, CandleBodyType.Close]

In [3]:
price_pct = .0075

In [4]:
closing_prices[0] + (closing_prices[0] * price_pct)

51655.9355

In [5]:
closing_prices[0] - (closing_prices[0] * price_pct)

50886.8645

In [6]:
sell_order_price = closing_prices[0] + (closing_prices[0] * price_pct)
sell_signals = np.full_like(closing_prices, np.nan)

buy_order_price = closing_prices[0] - (closing_prices[0] * price_pct)
buy_signals = np.full_like(closing_prices, np.nan)

cum_pnl = np.full_like(closing_prices, np.nan)
pnl = np.full_like(closing_prices, np.nan)

current_pnl = 0
total_pnl = 0
position_size_usdt = 0
average_entry = 0
buy_size_usdt = 10
sell_size_usdt = 12
limit_fee_pct = .0003

for idx in range(1, closing_prices.size):
    if low_prices[idx] <= buy_order_price:
        buy_signals[idx] = buy_order_price

        if position_size_usdt <= 0:
            position_size_usdt += buy_size_usdt
            if position_size_usdt >= 0:
                tp_size_asset = abs(position_size_usdt - buy_size_usdt) / buy_order_price
                average_entry = buy_order_price
            else:
                tp_size_asset = buy_size_usdt / buy_order_price

            no_fee_pnl = round((buy_order_price - average_entry) * tp_size_asset, 3)  # math checked
            fee_open = tp_size_asset * average_entry * limit_fee_pct  # math checked
            fee_close = tp_size_asset * buy_order_price * limit_fee_pct  # math checked
            fees_paid = fee_open + fee_close  # math checked
            current_pnl = round(no_fee_pnl - fees_paid, 3)
            pnl[idx] = current_pnl
            total_pnl += current_pnl
            cum_pnl[idx] = total_pnl

        else:
            position_size_usdt += buy_size_usdt
            average_entry = (position_size_usdt + buy_size_usdt) / (
                (position_size_usdt / average_entry) + (buy_order_price / buy_size_usdt)
            )

        buy_order_price = closing_prices[idx] - (closing_prices[idx] * price_pct)
        sell_order_price = closing_prices[idx] + (closing_prices[idx] * price_pct)

    elif high_prices[idx] >= sell_order_price:
        sell_signals[idx] = sell_order_price

        if position_size_usdt >= 0:
            position_size_usdt -= sell_size_usdt
            if position_size_usdt <= 0:
                tp_size_asset = abs(position_size_usdt + sell_size_usdt) / sell_order_price
                average_entry = sell_order_price
            else:
                tp_size_asset = sell_size_usdt / sell_order_price

            no_fee_pnl = round((sell_order_price - average_entry) * tp_size_asset, 3)  # math checked
            fee_open = tp_size_asset * average_entry * limit_fee_pct  # math checked
            fee_close = tp_size_asset * sell_order_price * limit_fee_pct  # math checked
            fees_paid = fee_open + fee_close  # math checked
            current_pnl = round(no_fee_pnl - fees_paid, 3)
            pnl[idx] = current_pnl
            total_pnl += current_pnl
            cum_pnl[idx] = total_pnl

        else:
            average_entry = abs(position_size_usdt + -sell_size_usdt) / (
                (-position_size_usdt / average_entry) + (sell_order_price / sell_size_usdt)
            )
            position_size_usdt -= sell_size_usdt

        buy_order_price = closing_prices[idx] - (closing_prices[idx] * price_pct)
        sell_order_price = closing_prices[idx] + (closing_prices[idx] * price_pct)

In [7]:
go.Figure(
    data=[
        go.Candlestick(
            x=datetimes,
            open=open_prices,
            high=high_prices,
            low=low_prices,
            close=closing_prices,
            name="Candles",
        ),
        go.Scatter(
            x=datetimes,
            y=buy_signals,
            mode="markers",
            name="Buy Signals",
            marker=dict(
                size=15,
                symbol="circle",
                color="#00F6FF",
                line=dict(
                    width=1,
                    color="DarkSlateGrey",
                ),
            ),
        ),
        go.Scatter(
            x=datetimes,
            y=sell_signals,
            mode="markers",
            name="Sell Signals",
            marker=dict(
                size=15,
                symbol="circle",
                color="yellow",
                line=dict(
                    width=1,
                    color="DarkSlateGrey",
                ),
            ),
        ),
    ]
).update_layout(height=800, xaxis_rangeslider_visible=False)